In [ ]:
!pip install tensorflow opencv-python tqdm patchify


In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
from patchify import patchify
#from metrics import dice_loss, dice_coef
#from train import load_dataset, create_dir


In [ ]:
def load_dataset(path, split=0.1):
    X = sorted(glob(os.path.join(path, "images", "*.png")))
    Y = sorted(glob(os.path.join(path, "masks", "*.png")))

    split_size = int(len(X) * split)

    train_x, valid_x = train_test_split(X, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(Y, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
""" UNETR Configuration """
cf = {
    "image_size": 256,
    "num_channels": 3,
    "num_layers": 12,
    "hidden_dim": 128,
    "mlp_dim": 32,
    "num_heads": 6,
    "dropout_rate": 0.1,
    "patch_size": 16,
    "num_patches": (256**2)//(16**2),
    "flat_patches_shape": (256**2//(16**2), 16*16*3)
}


In [ ]:
# Load the trained model
model_path = '/content/drive/MyDrive/your_model_path/model.h5'  # Adjust path to your model
model = tf.keras.models.load_model(model_path, custom_objects={"dice_loss": dice_loss, "dice_coef": dice_coef})


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/your_model_path/model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Create a directory for saving results
create_dir(f"/content/drive/MyDrive/results")

# Example test data (ensure you load this data properly from Google Drive)
test_x = ['/content/drive/MyDrive/your_data_path/images/img1.png', '/content/drive/MyDrive/your_data_path/images/img2.png']  # Example paths
test_y = ['/content/drive/MyDrive/your_data_path/masks/mask1.png', '/content/drive/MyDrive/your_data_path/masks/mask2.png']

# Prediction
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    # Extracting the image name
    name = x.split("/")[-1]

    # Reading the image
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    x = image / 255.0

    patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    patches = patchify(x, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)
    patches = np.expand_dims(patches, axis=0)

    # Read the mask
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    mask = mask / 255.0
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    # Prediction
    pred = model.predict(patches, verbose=0)[0]
    pred = np.concatenate([pred, pred, pred], axis=-1)

    # Saving the result (original image, mask, and prediction)
    line = np.ones((cf["image_size"], 10, 3)) * 255
    cat_images = np.concatenate([image, line, mask * 255, line, pred * 255], axis=1)

    # Save the final image
    save_image_path = os.path.join("/content/drive/MyDrive/results", name)
    cv2.imwrite(save_image_path, cat_images)


  0%|          | 0/2 [00:00<?, ?it/s]


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
